In [1]:
! pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=0bde2e625d994892c45e115c7cf9135f404b0bb8bea61404d06798ca96af58eb
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,expr,avg,sum,max

spark=SparkSession.builder \
  .appName("EcommerceTransactions") \
  .getOrCreate()

data = [
    (1, 101, 'Laptop', 'Electronics', 1000, 1, 10, '2023-08-01'),
    (2, 102, 'Smartphone', 'Electronics', 700, 2, 5, '2023-08-01'),
    (3, 103, 'Shirt', 'Fashion', 40, 3, 0, '2023-08-02'),
    (4, 104, 'Blender', 'Home Appliance', 150, 1, 15, '2023-08-03'),
    (5, 101, 'Headphones', 'Electronics', 100, 2, 10, '2023-08-03'),
    (6, 105, 'Shoes', 'Fashion', 60, 1, 20, '2023-08-04'),
    (7, 106, 'Refrigerator', 'Home Appliance', 800, 1, 25, '2023-08-05'),
    (8, 107, 'Book', 'Books', 20, 4, 0, '2023-08-05'),
    (9, 108, 'Toaster', 'Home Appliance', 30, 1, 5, '2023-08-06'),
    (10, 102, 'Tablet', 'Electronics', 300, 2, 10, '2023-08-06')
]

columns = ['transaction_id', 'customer_id', 'product', 'category', 'price', 'quantity', 'discount_percentage', 'transaction_date']
df=spark.createDataFrame(data,columns)

# 1. Calculate the Total Revenue per Category
df = df.withColumn("revenue", col("price") * col("quantity") * (1 - col("discount_percentage") / 100))
total_revenue_per_category = df.groupBy("category").agg(sum("revenue").alias("total_revenue"))
print("Total Revenue per Category:")
total_revenue_per_category.show()

# 2. Filter Transactions with a Discount Greater Than 10%
discount_gt_10 = df.filter(col("discount_percentage") > 10)
print("Transactions with a Discount Greater Than 10%:")
discount_gt_10.show()

# 3. Find the Most Expensive Product Sold
most_expensive_product = df.orderBy(col("price").desc()).limit(1)
print("Most Expensive Product Sold:")
most_expensive_product.show()

# 4. Calculate the Average Quantity of Products Sold per Category
avg_quantity_per_category = df.groupBy("category").agg(avg("quantity").alias("avg_quantity"))
print("Average Quantity of Products Sold per Category:")
avg_quantity_per_category.show()

# 5. Identify Customers Who Purchased More Than One Product in a Single Transaction
multiple_products = df.filter(col("quantity") > 1)
print("Customers Who Purchased More Than One Product in a Single Transaction:")
multiple_products.show()

# 6. Find the Top 3 Highest Revenue Transactions
top_3_transactions = df.orderBy(col("revenue").desc()).limit(3)
print("Top 3 Highest Revenue Transactions:")
top_3_transactions.show()

# 7. Calculate the Total Number of Transactions per Day
transactions_per_day = df.groupBy("transaction_date").count().alias("total_transactions")
print("Total Number of Transactions per Day:")
transactions_per_day.show()

# 8. Find the Customer Who Spent the Most Money
total_spent_per_customer = df.groupBy("customer_id").agg(sum("revenue").alias("total_spent"))
top_spending_customer = total_spent_per_customer.orderBy(col("total_spent").desc()).limit(1)
print("Customer Who Spent the Most Money:")
top_spending_customer.show()

# 9. Calculate the Average Discount Given per Product Category
avg_discount_per_category = df.groupBy("category").agg(avg("discount_percentage").alias("avg_discount"))
print("Average Discount Given per Product Category:")
avg_discount_per_category.show()

# 10. Create a New Column for Final Price After Discount
df = df.withColumn("final_price", col("price") * (1 - col("discount_percentage") / 100))
print("New Column for Final Price After Discount:")
df.select("product", "final_price").show()

Total Revenue per Category:
+--------------+-------------+
|      category|total_revenue|
+--------------+-------------+
|       Fashion|        168.0|
|   Electronics|       2950.0|
|Home Appliance|        756.0|
|         Books|         80.0|
+--------------+-------------+

Transactions with a Discount Greater Than 10%:
+--------------+-----------+------------+--------------+-----+--------+-------------------+----------------+-------+
|transaction_id|customer_id|     product|      category|price|quantity|discount_percentage|transaction_date|revenue|
+--------------+-----------+------------+--------------+-----+--------+-------------------+----------------+-------+
|             4|        104|     Blender|Home Appliance|  150|       1|                 15|      2023-08-03|  127.5|
|             6|        105|       Shoes|       Fashion|   60|       1|                 20|      2023-08-04|   48.0|
|             7|        106|Refrigerator|Home Appliance|  800|       1|                 25|

In [11]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,expr,avg,sum,max,count,when

spark=SparkSession.builder \
  .appName("BankingTransactions") \
  .getOrCreate()

data = [
    (1, 201, 'Deposit', 5000.0, '2023-09-01'),
    (2, 202, 'Withdrawal', 2000.0, '2023-09-01'),
    (3, 203, 'Deposit', 3000.0, '2023-09-02'),
    (4, 201, 'Withdrawal', 1500.0, '2023-09-02'),
    (5, 204, 'Deposit', 10000.0, '2023-09-03'),
    (6, 205, 'Withdrawal', 500.0, '2023-09-03'),
    (7, 202, 'Deposit', 2500.0, '2023-09-04'),
    (8, 206, 'Withdrawal', 700.0, '2023-09-04'),
    (9, 203, 'Deposit', 4000.0, '2023-09-05'),
    (10, 204, 'Withdrawal', 3000.0, '2023-09-05')
]

columns = ["transaction_id","customer_id","transaction_type","amount","transaction_date"]
df=spark.createDataFrame(data,columns)
df.show()

# 1. Calculate the Total Deposit and Withdrawal Amounts Group the data by transaction_type and calculate the total amounts for both deposits and withdrawals.
total_amount=df.groupBy("transaction_type").agg(sum("amount").alias("total_amount"))
print("Calculate the Total Deposit and Withdrawal Amounts")
total_amount.show()

#2. Filter Transactions Greater Than $3,000 Filter the dataset to show only transactions where the amount is greater than $3,000.
transactions_above_3000 = df.filter(col("amount")>3000)
print("Filter Transactions Greater Than $3,000")
transactions_above_3000.show()

#3. Find the Largest Deposit Made Identify the transaction with the highest deposit amount.
largest_deposit = df.filter(col("transaction_type") == "Deposit").agg(max("amount").alias("largest_deposit"))
print("Find the Largest Deposit Made")
largest_deposit.show()

# 4. Calculate the Average Transaction Amount for Each Transaction Type Group the data by transaction_type and calculate the average amount for deposits and withdrawals.
avg_transaction_amount=df.groupBy("transaction_type").agg(avg("amount").alias("average_amount"))
print("Calculate the Average Transaction Amount for Each Transaction Type")
avg_transaction_amount.show()

# 5. Find Customers Who Made Both Deposits and Withdrawals Identify customers who have made at least one deposit and one withdrawal.
depo_customers=df.filter(col("transaction_type")=="Deposit").select("customer_id").distinct()
withdraw_customers= df.filter(col("transaction_type")=="Withdrawal").select("customer_id").distinct()
customers_both = depo_customers.intersect(withdraw_customers)
print("Find Customers Who Made Both Deposits and Withdrawals")
customers_both.show()

# 6. Calculate the Total Amount of Transactions per Day Group the data by transaction_date and calculate the total amount of all transactions for each day.
total_amount_per_day = df.groupBy("transaction_date").agg(sum("amount").alias("total_amount"))
print("Calculate the Total Amount of Transactions per Day")
total_amount_per_day.show()

# 7. Find the Customer with the Highest Total Withdrawal Calculate the total amount withdrawn by each customer and identify the customer with the highest total withdrawal.
total_withdrawals = df.filter(col("transaction_type")=="Withdrawal").groupBy("customer_id").agg(sum("amount").alias("total_withdrawal"))
highest_withdrawal_customer = total_withdrawals.orderBy(col("total_withdrawal").desc()).limit(1)
print("Find the Customer with the Highest Total Withdrawal")
highest_withdrawal_customer.show()

# 8. Calculate the Number of Transactions for Each Customer Group the data by customer_id and calculate the total number of transactions made by each customer.
transactions_count_per_customer = df.groupBy("customer_id").agg(count("transaction_id").alias("transaction_count"))
print("Calculate the Number of Transactions for Each Customer")
transactions_count_per_customer.show()

# 9. Find All Transactions That Occurred on the Same Day as a Withdrawal Greater Than $1,000 Filter the data to show all transactions that occurred on the same day as a withdrawal of more than $1,000.
withdrawal_days = df.filter((col("transaction_type") == "Withdrawal") & (col("amount") > 1000)).select("transaction_date").distinct()
transactions_on_withdrawal_days = df.join(withdrawal_days, "transaction_date", "inner")
print("Find All Transactions That Occurred on the Same Day as a Withdrawal Greater Than $1,000")
transactions_on_withdrawal_days.show()

# 10. Create a New Column to Classify Transactions as "High" or "Low" Value Add a new column transaction_value that classifies a transaction as "High" if the amount is greater than $5,000, otherwise classify it as "Low."
df_with_value = df.withColumn("transaction_value", when(col("amount") > 5000, "High").otherwise("Low"))
print("Create a New Column to Classify Transactions as 'High' or 'Low' Value")
df_with_value.show()

+--------------+-----------+----------------+-------+----------------+
|transaction_id|customer_id|transaction_type| amount|transaction_date|
+--------------+-----------+----------------+-------+----------------+
|             1|        201|         Deposit| 5000.0|      2023-09-01|
|             2|        202|      Withdrawal| 2000.0|      2023-09-01|
|             3|        203|         Deposit| 3000.0|      2023-09-02|
|             4|        201|      Withdrawal| 1500.0|      2023-09-02|
|             5|        204|         Deposit|10000.0|      2023-09-03|
|             6|        205|      Withdrawal|  500.0|      2023-09-03|
|             7|        202|         Deposit| 2500.0|      2023-09-04|
|             8|        206|      Withdrawal|  700.0|      2023-09-04|
|             9|        203|         Deposit| 4000.0|      2023-09-05|
|            10|        204|      Withdrawal| 3000.0|      2023-09-05|
+--------------+-----------+----------------+-------+----------------+

Calcu

In [18]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, avg, max, when, countDistinct, count, row_number
from pyspark.sql.window import Window

# Initialize Spark session
spark = SparkSession.builder \
        .appName("HealthFitnessTracker") \
        .getOrCreate()

# Sample data
data = [
    (1, '2023-09-01', 12000, 500, 7.0, 'Cardio'),
    (2, '2023-09-01', 8000, 400, 6.5, 'Strength'),
    (3, '2023-09-01', 15000, 650, 8.0, 'Yoga'),
    (1, '2023-09-02', 10000, 450, 6.0, 'Cardio'),
    (2, '2023-09-02', 9500, 500, 7.0, 'Cardio'),
    (3, '2023-09-02', 14000, 600, 7.5, 'Strength'),
    (1, '2023-09-03', 13000, 550, 8.0, 'Yoga'),
    (2, '2023-09-03', 12000, 520, 6.5, 'Yoga'),
    (3, '2023-09-03', 16000, 700, 7.0, 'Cardio')
]


columns = ["user_id", "date", "steps", "calories_burned", "hours_of_sleep", "workout_type"]
df = spark.createDataFrame(data, schema=columns)

# 1. Find the Total Steps Taken by Each User
total_steps_per_user = df.groupBy("user_id").agg(sum("steps").alias("total_steps"))
print("Total Steps Taken by Each User")
total_steps_per_user.show()

# 2. Filter Days with More Than 10,000 Steps
days_above_10000_steps = df.filter(df.steps > 10000)
print("Days with More Than 10,000 Steps")
days_above_10000_steps.show()

# 3. Calculate the Average Calories Burned by Workout Type
avg_calories_per_workout = df.groupBy("workout_type").agg(avg("calories_burned").alias("average_calories"))
print("Average Calories Burned by Workout Type")
avg_calories_per_workout.show()

# 4. Identify the Day with the Most Steps for Each User
window_spec = Window.partitionBy("user_id").orderBy(df.steps.desc())
most_steps_per_user = df.withColumn("rank", row_number().over(window_spec)).filter("rank = 1").drop("rank")
print("Day with the Most Steps for Each User")
most_steps_per_user.show()

# 5. Find Users Who Burned More Than 600 Calories on Any Day
users_above_600_calories = df.filter(df.calories_burned > 600).select("user_id").distinct()
print("Users Who Burned More Than 600 Calories on Any Day")
users_above_600_calories.show()

# 6. Calculate the Average Hours of Sleep per User
avg_sleep_per_user = df.groupBy("user_id").agg(avg("hours_of_sleep").alias("average_sleep"))
print("Average Hours of Sleep per User")
avg_sleep_per_user.show()

# 7. Find the Total Calories Burned per Day
total_calories_per_day = df.groupBy("date").agg(sum("calories_burned").alias("total_calories"))
("Total Calories Burned per Day")
total_calories_per_day.show()

# 8. Identify Users Who Did Different Types of Workouts
workout_types_per_user = df.groupBy("user_id").agg(countDistinct("workout_type").alias("workout_types_count"))
users_with_multiple_workouts = workout_types_per_user.filter(workout_types_per_user.workout_types_count > 1)
print("Users Who Did Different Types of Workouts")
users_with_multiple_workouts.show()

# 9. Calculate the Total Number of Workouts per User
total_workouts_per_user = df.groupBy("user_id").agg(count("workout_type").alias("total_workouts"))
print("Total Number of Workouts per User")
total_workouts_per_user.show()

# 10. Create a New Column for "Active" Days
df_with_active_day = df.withColumn("active_day", when(df.steps > 10000, "Active").otherwise("Inactive"))
print("New Column for 'Active' Days")
df_with_active_day.show()


Total Steps Taken by Each User
+-------+-----------+
|user_id|total_steps|
+-------+-----------+
|      1|      35000|
|      3|      45000|
|      2|      29500|
+-------+-----------+

Days with More Than 10,000 Steps
+-------+----------+-----+---------------+--------------+------------+
|user_id|      date|steps|calories_burned|hours_of_sleep|workout_type|
+-------+----------+-----+---------------+--------------+------------+
|      1|2023-09-01|12000|            500|           7.0|      Cardio|
|      3|2023-09-01|15000|            650|           8.0|        Yoga|
|      3|2023-09-02|14000|            600|           7.5|    Strength|
|      1|2023-09-03|13000|            550|           8.0|        Yoga|
|      2|2023-09-03|12000|            520|           6.5|        Yoga|
|      3|2023-09-03|16000|            700|           7.0|      Cardio|
+-------+----------+-----+---------------+--------------+------------+

Average Calories Burned by Workout Type
+------------+----------------

In [21]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, avg, max, count, row_number, col, to_date, countDistinct, when,desc
from pyspark.sql.window import Window

# Initialize Spark session
spark = SparkSession.builder \
        .appName("MusicStreamingData") \
        .getOrCreate()

# Sample data
data = [
    (1, "Blinding Lights", "The Weeknd", 200, "2023-09-01 08:15:00", "New York"),
    (2, "Shape of You", "Ed Sheeran", 240, "2023-09-01 09:20:00", "Los Angeles"),
    (3, "Levitating", "Dua Lipa", 180, "2023-09-01 10:30:00", "London"),
    (1, "Starboy", "The Weeknd", 220, "2023-09-01 11:00:00", "New York"),
    (2, "Perfect", "Ed Sheeran", 250, "2023-09-01 12:15:00", "Los Angeles"),
    (3, "Don't Start Now", "Dua Lipa", 200, "2023-09-02 08:10:00", "London"),
    (1, "Save Your Tears", "The Weeknd", 210, "2023-09-02 09:00:00", "New York"),
    (2, "Galway Girl", "Ed Sheeran", 190, "2023-09-02 10:00:00", "Los Angeles"),
    (3, "New Rules", "Dua Lipa", 230, "2023-09-02 11:00:00", "London")
]


columns = ["user_id", "song_title", "artist", "duration_seconds", "streaming_time", "location"]
df = spark.createDataFrame(data, schema=columns)

# 1. Calculate the Total Listening Time for Each User
total_listening_time = df.groupBy("user_id").agg(sum("duration_seconds").alias("total_listening_time"))
print("Total Listening Time for Each User")
total_listening_time.show()

# 2. Filter Songs Streamed for More Than 200 Seconds
songs_more_than_200 = df.filter(col("duration_seconds")> 200)
print("Songs Streamed for More Than 200 Seconds")
songs_more_than_200.show()

# 3. Find the Most Popular Artist (by Total Streams)
popular_artist = df.groupBy("artist").agg(count("song_title").alias("total_streams")).orderBy(col("total_streams").desc()).limit(1)
print("Most Popular Artist")
popular_artist.show()

# 4. Identify the Song with the Longest Duration
longest_song = df.orderBy(col("duration_seconds").desc()).limit(1)
print("Song with the Longest Duration")
longest_song.show()

# 5. Calculate the Average Song Duration by Artist
avg_song_duration_by_artist = df.groupBy("artist").agg(avg("duration_seconds").alias("average_duration"))
print("Average Song Duration by Artist")
avg_song_duration_by_artist.show()

# 6. Find the Top 3 Most Streamed Songs per User
song_stream_counts = df.groupBy("user_id", "song_title").agg(count("*").alias("stream_count"))
window_spec = Window.partitionBy("user_id").orderBy(desc("stream_count"))
ranked_songs = song_stream_counts.withColumn("rank", row_number().over(window_spec))
top_3_songs_per_user = ranked_songs.filter(col("rank") <= 3).select("user_id", "song_title", "stream_count")
top_3_songs_per_user.show

# 7. Calculate the Total Number of Streams per Day
total_streams_per_day = df.withColumn("stream_date", to_date(col("streaming_time"))).groupBy("stream_date").agg(count("song_title").alias("total_streams"))
print("Total Number of Streams per Day")
total_streams_per_day.show()

# 8. Identify Users Who Streamed Songs from More Than One Artist
users_multiple_artists = df.groupBy("user_id").agg(countDistinct("artist").alias("artist_count")).filter(col("artist_count") > 1)
print("Users Who Streamed Songs from More Than One Artist")
users_multiple_artists.show()

# 9. Calculate the Total Streams for Each Location
total_streams_by_location = df.groupBy("location").agg(count("song_title").alias("total_streams"))
print("Total Streams for Each Location")
total_streams_by_location.show()

# 10. Create a New Column to Classify Long and Short Songs
df_with_song_length = df.withColumn("song_length", when(df.duration_seconds > 200, "Long").otherwise("Short"))
print("a New Column to Classify Long and Short Songs")
df_with_song_length.show()


Total Listening Time for Each User
+-------+--------------------+
|user_id|total_listening_time|
+-------+--------------------+
|      1|                 630|
|      3|                 610|
|      2|                 680|
+-------+--------------------+

Songs Streamed for More Than 200 Seconds
+-------+---------------+----------+----------------+-------------------+-----------+
|user_id|     song_title|    artist|duration_seconds|     streaming_time|   location|
+-------+---------------+----------+----------------+-------------------+-----------+
|      2|   Shape of You|Ed Sheeran|             240|2023-09-01 09:20:00|Los Angeles|
|      1|        Starboy|The Weeknd|             220|2023-09-01 11:00:00|   New York|
|      2|        Perfect|Ed Sheeran|             250|2023-09-01 12:15:00|Los Angeles|
|      1|Save Your Tears|The Weeknd|             210|2023-09-02 09:00:00|   New York|
|      3|      New Rules|  Dua Lipa|             230|2023-09-02 11:00:00|     London|
+-------+---------

In [22]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, desc, row_number
from pyspark.sql.window import Window

# Initialize Spark session
spark = SparkSession.builder.appName("RetailStoreSalesData").getOrCreate()

# Sample data
data = [
    (1, "Apple", "Groceries", 0.50, 10, "2023-09-01"),
    (2, "T-shirt", "Clothing", 15.00, 2, "2023-09-01"),
    (3, "Notebook", "Stationery", 2.00, 5, "2023-09-02"),
    (4, "Banana", "Groceries", 0.30, 12, "2023-09-02"),
    (5, "Laptop", "Electronics", 800.00, 1, "2023-09-03"),
    (6, "Pants", "Clothing", 25.00, 3, "2023-09-03"),
    (7, "Headphones", "Electronics", 100.00, 2, "2023-09-04"),
    (8, "Pen", "Stationery", 1.00, 10, "2023-09-04"),
    (9, "Orange", "Groceries", 0.60, 8, "2023-09-05"),
    (10, "Sneakers", "Clothing", 50.00, 1, "2023-09-05")
]

columns = ["transaction_id", "product_name", "category", "price", "quantity", "sales_date"]
df = spark.createDataFrame(data, schema=columns)


df_with_revenue = df.withColumn("revenue", col("price") * col("quantity"))

# 1. Calculate the Total Revenue per Category
total_revenue_per_category = df_with_revenue.groupBy("category").agg(sum("revenue").alias("total_revenue"))
print("Total Revenue per Category")
total_revenue_per_category.show()

# 2. Filter Transactions Where the Total Sales Amount is Greater Than $100
filtered_transactions = df_with_revenue.filter(col("revenue") > 100)
print("Transactions Where the Total Sales Amount is Greater Than $100")
filtered_transactions.show()

# 3. Find the Most Sold Product
total_quantity_per_product = df.groupBy("product_name").agg(sum("quantity").alias("total_quantity"))
most_sold_product = total_quantity_per_product.orderBy(desc("total_quantity")).first()
print("Most Sold Product:", most_sold_product)

# 4. Calculate the Average Price per Product Category
average_price_per_category = df.groupBy("category").agg(sum("price").alias("total_price"), sum("quantity").alias("total_quantity"))
average_price_per_category = average_price_per_category.withColumn("average_price", col("total_price") / col("total_quantity"))
print("Average Price per Product Category")
average_price_per_category.select("category", "average_price").show()

# 5. Find the Top 3 Highest Grossing Products
top_3_grossing_products = total_revenue_per_category.orderBy(desc("total_revenue")).limit(3)
print("Top 3 Highest Grossing Products")
top_3_grossing_products.show()

# 6. Calculate the Total Number of Items Sold per Day
total_quantity_per_day = df.groupBy("sales_date").agg(sum("quantity").alias("total_quantity"))
print("Total Number of Items Sold per Day")
total_quantity_per_day.show()

# 7. Identify the Product with the Lowest Price in Each Category
window_spec = Window.partitionBy("category").orderBy("price")
ranked_products = df.withColumn("rank", row_number().over(window_spec))
lowest_price_per_category = ranked_products.filter(col("rank") == 1)
print("Product with the Lowest Price in Each Category")
lowest_price_per_category.show()

# 8. Calculate the Total Revenue for Each Product
total_revenue_per_product = df_with_revenue.groupBy("product_name").agg(sum("revenue").alias("total_revenue"))
print("Total Revenue for Each Product")
total_revenue_per_product.show()

# 9. Find the Total Sales per Day for Each Category
total_sales_per_day_category = df_with_revenue.groupBy("sales_date", "category").agg(sum("revenue").alias("total_sales"))
print("Total Sales per Day for Each Category")
total_sales_per_day_category.show()

# 10. Create a New Column for Discounted Price
df_with_discounted_price = df.withColumn("discounted_price", col("price") * 0.9)
print("New Column for Discounted Price")
df_with_discounted_price.show()


Total Revenue per Category
+-----------+------------------+
|   category|     total_revenue|
+-----------+------------------+
| Stationery|              20.0|
|  Groceries|13.399999999999999|
|Electronics|            1000.0|
|   Clothing|             155.0|
+-----------+------------------+

Transactions Where the Total Sales Amount is Greater Than $100
+--------------+------------+-----------+-----+--------+----------+-------+
|transaction_id|product_name|   category|price|quantity|sales_date|revenue|
+--------------+------------+-----------+-----+--------+----------+-------+
|             5|      Laptop|Electronics|800.0|       1|2023-09-03|  800.0|
|             7|  Headphones|Electronics|100.0|       2|2023-09-04|  200.0|
+--------------+------------+-----------+-----+--------+----------+-------+

Most Sold Product: Row(product_name='Banana', total_quantity=12)
Average Price per Product Category
+-----------+-------------------+
|   category|      average_price|
+-----------+-------